In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub

import PIL.Image as Image
import os
import cv2

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [2]:
m = Sequential([
    hub.KerasLayer("https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow2/035-128-classification/2")
])
m.build([None, 128, 128, 3])  # Batch input shape.

In [27]:
def image(img_path):
    img = Image.open(img_path).resize((128,128))
    img = np.array(img)/255
    img = img[np.newaxis, ...]
    return img

img_path = "goldfish.jpg"
goldfish = image(img_path)

In [28]:
goldfish.shape

(1, 128, 128, 3)

In [29]:
result = np.argmax(m.predict(goldfish))
result

1/1 [==============================] - 0s 20ms/step


2

In [18]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,cache_dir=".", untar=True)

228813984/228813984 [==============================] - 18s 0us/step


In [19]:
data_dir

'./datasets/flower_photos'

In [20]:
import pathlib

data_dir = pathlib.Path(data_dir)

# making a dictionary of directories
flower_dir = {
    'roses' : list(data_dir.glob("roses/*")),
    'daisy' : list(data_dir.glob("daisy/*")),
    'dandelion' : list(data_dir.glob("dandelion/*")),
    'sunflowers' : list(data_dir.glob("sunflowers/*")),
    'tulips' : list(data_dir.glob("tulips/*"))
}

flower_num = {
    'roses' : 0,
    'daisy' : 1,
    'dandelion' : 2,
    'sunflowers' : 3,
    'tulips' : 4
}

In [42]:
X, y = [], []

for flower_name, images in flower_dir.items():
    for image in images:
        img = cv2.imread(str(image))
        img = cv2.resize(img,(128,128))
        X.append(img)
        y.append(flower_num[flower_name])

In [43]:
X = np.array(X)/255
y = np.array(y)

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [61]:
X_train.shape

(2752, 128, 128, 3)

## Transfer Model

In [64]:
model =Sequential([
    hub.KerasLayer("https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow2/035-128-feature-vector/2",
                   trainable=False), # this will give feature vector now
    layers.Dense(5, activation='softmax') # classification of flowers only
])
model.build([None, 128, 128, 3])  # Batch input shape.

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_2 (KerasLayer)  (None, 1280)              410208    
                                                                 
 dense_1 (Dense)             (None, 5)                 6405      
                                                                 
Total params: 416613 (1.59 MB)
Trainable params: 6405 (25.02 KB)
Non-trainable params: 410208 (1.56 MB)
_________________________________________________________________


In [66]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
86/86 [==============================] - 5s 52ms/step - loss: 0.8991 - accuracy: 0.6610
Epoch 2/10
86/86 [==============================] - 4s 52ms/step - loss: 0.5469 - accuracy: 0.8016
Epoch 3/10
86/86 [==============================] - 4s 52ms/step - loss: 0.4507 - accuracy: 0.8376
Epoch 4/10
86/86 [==============================] - 4s 50ms/step - loss: 0.3892 - accuracy: 0.8685
Epoch 5/10
86/86 [==============================] - 4s 51ms/step - loss: 0.3478 - accuracy: 0.8899
Epoch 6/10
86/86 [==============================] - 4s 49ms/step - loss: 0.3112 - accuracy: 0.8964
Epoch 7/10
86/86 [==============================] - 4s 42ms/step - loss: 0.2812 - accuracy: 0.9150
Epoch 8/10
86/86 [==============================] - 4s 44ms/step - loss: 0.2565 - accuracy: 0.9208
Epoch 9/10
86/86 [==============================] - 4s 42ms/step - loss: 0.2380 - accuracy: 0.9251
Epoch 10/10
86/86 [==============================] - 4s 46ms/step - loss: 0.2216 - accuracy: 0.9353


In [67]:
model.evaluate(X_test,y_test)

/home/shravan/.local/lib/python3.10/site-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


29/29 [==============================] - 2s 45ms/step - loss: 0.5460 - accuracy: 0.8072


[0.5460209250450134, 0.8071895241737366]

## Conclusion
### we got better accuracy of 80% in just 10 epochs after using TRANSFER LEARNING